# Denario Manuscript Review and Completion Workflow

This notebook demonstrates how to use Denario for comprehensive manuscript review, including:

## Key Features:
- **LaTeX to JSONL Conversion** - Convert manuscripts to structured format using ragbook
- **Section Completeness Analysis** - Identify incomplete sections with semantic understanding
- **Rich Structured Analysis** - Process prose, equations, figures, and tables separately
- **Citation Integration** - Add relevant in-text citations with context
- **Mathematical Verification** - Check calculations and derivations with normalized math
- **Simulation Validation** - Verify computational results
- **Content Enhancement** - Improve flow and completeness
- **Referee Response** - Address reviewer comments systematically

## Workflow:
1. **Upload your manuscript** (LaTeX, Word, or Markdown)
2. **Convert LaTeX to JSONL** (Recommended) - Use ragbook for rich structured analysis
3. **Upload referee report** (if available) for systematic comment parsing
4. **Analyze completeness** and identify gaps using structured data
5. **Generate responses** to referee comments with priority handling
6. **Generate missing content** with proper citations
7. **Verify all mathematical derivations** and equations
8. **Validate simulations and calculations**
9. **Generate revised manuscript** with enhanced structure


In [7]:
# Import necessary libraries for manuscript review
from denario import Denario, Journal, models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import json
from pathlib import Path
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

print("📚 Manuscript Review System Initialized")
print("=" * 50)


📚 Manuscript Review System Initialized


## 1. Manuscript Upload and Analysis

### Option A: Direct Upload (LaTeX, Word, Markdown)
### Option B: Convert to JSONL with ragbook (Recommended for Rich Analysis)


In [6]:
# ragbook Conversion Interface
def create_ragbook_conversion_interface():
    """Create interactive interface for ragbook conversion"""
    
    # LaTeX file upload widget
    tex_upload = widgets.FileUpload(
        accept='.tex',
        multiple=False,
        description='Upload LaTeX File',
        style={'description_width': 'initial'}
    )
    
    # Source ID input
    source_id_input = widgets.Text(
        value='manuscript',
        placeholder='Enter source ID (e.g., my-paper-v1)',
        description='Source ID:',
        style={'description_width': 'initial'}
    )
    
    # Convert button
    convert_button = widgets.Button(
        description='Convert to JSONL',
        button_style='primary',
        icon='cogs'
    )
    
    # Output area
    conversion_output = widgets.Output()
    
    def on_convert_clicked(b):
        with conversion_output:
            conversion_output.clear_output()
            
            if not tex_upload.value:
                print("❌ Please upload a LaTeX file first")
                return
            
            # Get the uploaded file
            uploaded_file = list(tex_upload.value.values())[0]
            file_name = uploaded_file['name']
            
            # Save to temporary file
            temp_tex_path = f"/tmp/{file_name}"
            with open(temp_tex_path, 'wb') as f:
                f.write(uploaded_file['content'])
            
            print(f"📄 Processing LaTeX file: {file_name}")
            print("=" * 50)
            
            # Convert to JSONL
            source_id = source_id_input.value.strip() or 'manuscript'
            jsonl_path = convert_manuscript_to_jsonl(temp_tex_path, source_id=source_id)
            
            if jsonl_path:
                # Analyze conversion results
                analysis = analyze_conversion_results(jsonl_path)
                
                if analysis:
                    print(f"\n✅ Conversion completed successfully!")
                    print(f"📁 JSONL file: {jsonl_path}")
                    print(f"📊 Ready for rich structured analysis")
                    
                    # Store the JSONL path for later use
                    global current_jsonl_path
                    current_jsonl_path = jsonl_path
                    
                    print(f"\n💡 Next steps:")
                    print(f"  1. Use the 'Enhanced Analysis' section below")
                    print(f"  2. Upload the JSONL file for rich structured analysis")
                    print(f"  3. Or use the direct analysis with the JSONL parser")
                else:
                    print("❌ Failed to analyze conversion results")
            else:
                print("❌ Conversion failed")
    
    convert_button.on_click(on_convert_clicked)
    
    # Layout
    interface = widgets.VBox([
        widgets.HTML("<h3>🔄 ragbook LaTeX to JSONL Conversion</h3>"),
        widgets.HTML("<p>Convert your LaTeX manuscript to structured JSONL format for rich analysis:</p>"),
        widgets.HTML("<ul><li>📝 Prose chunks with semantic structure</li><li>🧮 Equations with normalized math</li><li>📊 Tables exported to CSV</li><li>🖼️ Figures with captions and metadata</li></ul>"),
        widgets.HBox([tex_upload, convert_button]),
        source_id_input,
        conversion_output
    ])
    
    return interface

# Create and display ragbook conversion interface
ragbook_interface = create_ragbook_conversion_interface()
display(ragbook_interface)


In [ ]:
# ragbook integration for LaTeX to JSONL conversion
import subprocess
import sys
import os
from pathlib import Path

def convert_manuscript_to_jsonl(tex_file_path, output_dir="/tmp", source_id="manuscript"):
    """
    Convert LaTeX manuscript to JSONL format using ragbook
    
    Args:
        tex_file_path: Path to the main .tex file
        output_dir: Directory to save the JSONL output
        source_id: Identifier for the source document
    
    Returns:
        Path to the generated JSONL file, or None if conversion failed
    """
    try:
        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        # Get the project root (directory containing the tex file)
        project_root = os.path.dirname(os.path.abspath(tex_file_path))
        tex_filename = os.path.basename(tex_file_path)
        
        # Output JSONL file path
        jsonl_output = os.path.join(output_dir, f"{source_id}.jsonl")
        
        # ragbook command
        cmd = [
            sys.executable,  # Use current Python interpreter
            "/data/cmbagents/ragbook/convert_book.py",
            "--entry", tex_filename,
            "--root", project_root,
            "--source-id", source_id,
            "--out", jsonl_output,
            "--assets-dir", project_root,  # Look for figures in the same directory
            "--assets-url-prefix", "assets/",
            "--no-ocr"  # Disable OCR for faster processing
        ]
        
        print(f"🔄 Converting {tex_filename} to JSONL format...")
        print(f"📁 Project root: {project_root}")
        print(f"📄 Output: {jsonl_output}")
        
        # Run the conversion
        result = subprocess.run(cmd, capture_output=True, text=True, cwd=project_root)
        
        if result.returncode == 0:
            print("✅ Conversion successful!")
            print(f"📊 JSONL file created: {jsonl_output}")
            
            # Check if file was created and has content
            if os.path.exists(jsonl_output) and os.path.getsize(jsonl_output) > 0:
                return jsonl_output
            else:
                print("❌ JSONL file was created but is empty")
                return None
        else:
            print(f"❌ Conversion failed with return code {result.returncode}")
            print(f"Error output: {result.stderr}")
            return None
            
    except Exception as e:
        print(f"❌ Error during conversion: {e}")
        return None

def analyze_conversion_results(jsonl_path):
    """Analyze the results of ragbook conversion"""
    try:
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Count different types of chunks
        chunk_types = {}
        total_chunks = len(lines)
        
        for line in lines:
            if line.strip():
                chunk = json.loads(line.strip())
                chunk_type = chunk.get('type', 'unknown')
                chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1
        
        print(f"📊 CONVERSION ANALYSIS")
        print(f"=" * 30)
        print(f"Total chunks: {total_chunks}")
        print(f"Chunk types:")
        for chunk_type, count in chunk_types.items():
            print(f"  • {chunk_type}: {count}")
        
        return {
            'total_chunks': total_chunks,
            'chunk_types': chunk_types,
            'jsonl_path': jsonl_path
        }
        
    except Exception as e:
        print(f"❌ Error analyzing conversion results: {e}")
        return None

# JSONL parser for ragbook format
class JSONLManuscriptParser:
    """Parse ragbook JSONL format for manuscript analysis"""
    
    def __init__(self):
        self.chunks = []
        self.sections = {}
        self.equations = []
        self.figures = []
        self.tables = []
        
    def load_jsonl(self, file_path):
        """Load and parse JSONL file from ragbook"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        chunk = json.loads(line.strip())
                        self.chunks.append(chunk)
            
            # Organize chunks by type
            self._organize_chunks()
            print(f"✅ JSONL loaded: {len(self.chunks)} chunks, {len(self.sections)} sections")
            return True
            
        except Exception as e:
            print(f"❌ Error loading JSONL: {e}")
            return False
    
    def _organize_chunks(self):
        """Organize chunks by type and section"""
        for chunk in self.chunks:
            chunk_type = chunk.get('type', 'other')
            section_path = chunk.get('section_path', [])
            
            # Group by section
            section_key = ' > '.join(section_path) if section_path else 'Unknown'
            if section_key not in self.sections:
                self.sections[section_key] = {
                    'prose': [],
                    'equations': [],
                    'figures': [],
                    'tables': [],
                    'other': []
                }
            
            # Categorize by type
            if chunk_type == 'prose':
                self.sections[section_key]['prose'].append(chunk)
            elif chunk_type == 'equation':
                self.sections[section_key]['equations'].append(chunk)
                self.equations.append(chunk)
            elif chunk_type == 'figure':
                self.sections[section_key]['figures'].append(chunk)
                self.figures.append(chunk)
            elif chunk_type == 'table':
                self.sections[section_key]['tables'].append(chunk)
                self.tables.append(chunk)
            else:
                self.sections[section_key]['other'].append(chunk)
    
    def get_section_text(self, section_name):
        """Get all prose text for a section"""
        for section_key, content in self.sections.items():
            if section_name.lower() in section_key.lower():
                prose_chunks = content['prose']
                return ' '.join([chunk.get('text', '') for chunk in prose_chunks])
        return ""
    
    def get_equations_for_section(self, section_name):
        """Get all equations for a section"""
        equations = []
        for section_key, content in self.sections.items():
            if section_name.lower() in section_key.lower():
                for eq_chunk in content['equations']:
                    equations.append({
                        'tex': eq_chunk.get('equation_tex', ''),
                        'normalized': eq_chunk.get('math_norm', ''),
                        'number': eq_chunk.get('eq_number', ''),
                        'text': eq_chunk.get('text', '')
                    })
        return equations
    
    def get_figures_for_section(self, section_name):
        """Get all figures for a section"""
        figures = []
        for section_key, content in self.sections.items():
            if section_name.lower() in section_key.lower():
                for fig_chunk in content['figures']:
                    figures.append({
                        'caption': fig_chunk.get('caption_tex', ''),
                        'filename': fig_chunk.get('filename', ''),
                        'asset_uri': fig_chunk.get('asset_uri', ''),
                        'text': fig_chunk.get('text', '')
                    })
        return figures
    
    def get_tables_for_section(self, section_name):
        """Get all tables for a section"""
        tables = []
        for section_key, content in self.sections.items():
            if section_name.lower() in section_key.lower():
                for table_chunk in content['tables']:
                    tables.append({
                        'caption': table_chunk.get('caption_tex', ''),
                        'csv_uri': table_chunk.get('csv_uri', ''),
                        'headers': table_chunk.get('headers', []),
                        'text': table_chunk.get('text', '')
                    })
        return tables
    
    def get_manuscript_summary(self):
        """Get comprehensive manuscript summary"""
        summary = {
            'total_chunks': len(self.chunks),
            'sections': len(self.sections),
            'total_equations': len(self.equations),
            'total_figures': len(self.figures),
            'total_tables': len(self.tables),
            'section_breakdown': {}
        }
        
        for section_key, content in self.sections.items():
            summary['section_breakdown'][section_key] = {
                'prose_chunks': len(content['prose']),
                'equations': len(content['equations']),
                'figures': len(content['figures']),
                'tables': len(content['tables'])
            }
        
        return summary

# Create manuscript analysis functions
class ManuscriptReviewer:
    def __init__(self, denario_instance):
        self.den = denario_instance
        self.manuscript_text = ""
        self.sections = {}
        self.issues = []
        self.citations = []
        
    def load_manuscript(self, file_path):
        """Load manuscript from file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                self.manuscript_text = f.read()
            print(f"✅ Manuscript loaded: {file_path}")
            return True
        except Exception as e:
            print(f"❌ Error loading manuscript: {e}")
            return False
    
    def analyze_sections(self):
        """Analyze manuscript sections for completeness"""
        # Common section patterns
        section_patterns = {
            'Abstract': r'\\abstract\{(.*?)\}',
            'Introduction': r'\\section\{Introduction\}(.*?)(?=\\section|\\end)',
            'Methods': r'\\section\{Methodology?\}(.*?)(?=\\section|\\end)',
            'Results': r'\\section\{Results?\}(.*?)(?=\\section|\\end)',
            'Discussion': r'\\section\{Discussion\}(.*?)(?=\\section|\\end)',
            'Conclusion': r'\\section\{Conclusion\}(.*?)(?=\\section|\\end)'
        }
        
        self.sections = {}
        for section_name, pattern in section_patterns.items():
            matches = re.findall(pattern, self.manuscript_text, re.DOTALL | re.IGNORECASE)
            if matches:
                self.sections[section_name] = matches[0].strip()
            else:
                self.sections[section_name] = ""
                self.issues.append(f"Missing or incomplete {section_name} section")
        
        return self.sections
    
    def check_citations(self):
        """Check for citation completeness"""
        # Find citation patterns
        citation_patterns = [
            r'\\cite\{[^}]+\}',  # LaTeX citations
            r'\[[0-9]+\]',       # Numbered citations
            r'\([A-Za-z]+ et al\., [0-9]{4}\)'  # Author-year citations
        ]
        
        found_citations = []
        for pattern in citation_patterns:
            found_citations.extend(re.findall(pattern, self.manuscript_text))
        
        # Check for uncited claims (simple heuristic)
        sentences = re.split(r'[.!?]+', self.manuscript_text)
        uncited_sentences = []
        
        for sentence in sentences:
            if len(sentence.strip()) > 50:  # Substantial sentence
                has_citation = any(pattern in sentence for pattern in citation_patterns)
                if not has_citation and any(word in sentence.lower() for word in 
                    ['show', 'demonstrate', 'prove', 'indicate', 'suggest', 'reveal']):
                    uncited_sentences.append(sentence.strip())
        
        self.citations = found_citations
        if uncited_sentences:
            self.issues.append(f"Found {len(uncited_sentences)} potentially uncited claims")
        
        return {
            'total_citations': len(found_citations),
            'uncited_claims': len(uncited_sentences),
            'uncited_sentences': uncited_sentences[:5]  # Show first 5
        }
    
    def check_mathematics(self):
        """Check mathematical content for completeness"""
        math_patterns = {
            'equations': r'\\begin\{equation\}(.*?)\\end\{equation\}',
            'inline_math': r'\$([^$]+)\$',
            'display_math': r'\$\$([^$]+)\$\$',
            'proofs': r'\\begin\{proof\}(.*?)\\end\{proof\}',
            'theorems': r'\\begin\{theorem\}(.*?)\\end\{theorem\}'
        }
        
        math_content = {}
        for math_type, pattern in math_patterns.items():
            matches = re.findall(pattern, self.manuscript_text, re.DOTALL)
            math_content[math_type] = matches
        
        # Check for incomplete derivations
        incomplete_derivations = []
        for eq in math_content['equations']:
            if '...' in eq or '\\ldots' in eq:
                incomplete_derivations.append(eq[:100] + "...")
        
        if incomplete_derivations:
            self.issues.append(f"Found {len(incomplete_derivations)} potentially incomplete derivations")
        
        return {
            'equations': len(math_content['equations']),
            'inline_math': len(math_content['inline_math']),
            'proofs': len(math_content['proofs']),
            'theorems': len(math_content['theorems']),
            'incomplete_derivations': incomplete_derivations
        }
    
    def generate_report(self):
        """Generate comprehensive analysis report"""
        sections = self.analyze_sections()
        citations = self.check_citations()
        math = self.check_mathematics()
        
        report = {
            'sections': sections,
            'citations': citations,
            'mathematics': math,
            'issues': self.issues,
            'total_issues': len(self.issues)
        }
        
        return report

# Initialize the reviewer
den = Denario(project_dir="manuscript_review")
reviewer = ManuscriptReviewer(den)

print("🔍 Manuscript Reviewer initialized")
print("Ready to analyze your manuscript!")


🔍 Manuscript Reviewer initialized
Ready to analyze your manuscript!


In [4]:
# JSONL analysis functions
def generate_jsonl_analysis_report(jsonl_parser):
    """Generate analysis report for JSONL manuscript"""
    summary = jsonl_parser.get_manuscript_summary()
    
    # Analyze section completeness
    sections = {}
    section_names = ['Abstract', 'Introduction', 'Methods', 'Results', 'Discussion', 'Conclusion']
    
    for section_name in section_names:
        section_text = jsonl_parser.get_section_text(section_name)
        sections[section_name] = section_text
        
    # Count citations (simple heuristic)
    total_citations = 0
    for chunk in jsonl_parser.chunks:
        if chunk.get('type') == 'prose':
            text = chunk.get('text', '')
            # Count various citation patterns
            citations = len(re.findall(r'\\cite\{[^}]+\}', text))
            citations += len(re.findall(r'\[[0-9]+\]', text))
            citations += len(re.findall(r'\([A-Za-z]+ et al\., [0-9]{4}\)', text))
            total_citations += citations
    
    # Analyze mathematics
    math_content = {
        'equations': len(jsonl_parser.equations),
        'inline_math': 0,  # Would need to count inline math in prose chunks
        'proofs': 0,
        'theorems': 0,
        'incomplete_derivations': []
    }
    
    # Check for incomplete derivations
    for eq in jsonl_parser.equations:
        eq_tex = eq.get('equation_tex', '')
        if '...' in eq_tex or '\\ldots' in eq_tex:
            math_content['incomplete_derivations'].append(eq_tex[:100] + "...")
    
    # Identify issues
    issues = []
    for section_name in section_names:
        if not sections[section_name].strip():
            issues.append(f"Missing or incomplete {section_name} section")
    
    if math_content['incomplete_derivations']:
        issues.append(f"Found {len(math_content['incomplete_derivations'])} potentially incomplete derivations")
    
    return {
        'sections': sections,
        'citations': {
            'total_citations': total_citations,
            'uncited_claims': 0,  # Would need more sophisticated analysis
            'uncited_sentences': []
        },
        'mathematics': math_content,
        'issues': issues,
        'total_issues': len(issues),
        'jsonl_summary': summary
    }

def display_jsonl_analysis_results(report):
    """Display analysis results for JSONL manuscript"""
    print("📊 JSONL MANUSCRIPT ANALYSIS REPORT")
    print("=" * 50)
    
    # JSONL summary
    if 'jsonl_summary' in report:
        summary = report['jsonl_summary']
        print(f"\n📄 DOCUMENT STRUCTURE:")
        print(f"  Total chunks: {summary['total_chunks']}")
        print(f"  Sections: {summary['sections']}")
        print(f"  Equations: {summary['total_equations']}")
        print(f"  Figures: {summary['total_figures']}")
        print(f"  Tables: {summary['total_tables']}")
    
    # Section completeness
    print("\n📝 SECTION COMPLETENESS:")
    for section, content in report['sections'].items():
        status = "✅" if content.strip() else "❌"
        length = len(content.split()) if content else 0
        print(f"  {status} {section}: {length} words")
    
    # Citation analysis
    print(f"\n📚 CITATION ANALYSIS:")
    print(f"  Total citations found: {report['citations']['total_citations']}")
    
    # Mathematical content
    print(f"\n🧮 MATHEMATICAL CONTENT:")
    print(f"  Equations: {report['mathematics']['equations']}")
    print(f"  Inline math: {report['mathematics']['inline_math']}")
    print(f"  Proofs: {report['mathematics']['proofs']}")
    print(f"  Theorems: {report['mathematics']['theorems']}")
    
    if report['mathematics']['incomplete_derivations']:
        print(f"  ⚠️  Incomplete derivations: {len(report['mathematics']['incomplete_derivations'])}")
    
    # Issues summary
    print(f"\n⚠️  ISSUES FOUND: {report['total_issues']}")
    for i, issue in enumerate(report['issues'], 1):
        print(f"  {i}. {issue}")

print("🔧 JSONL analysis functions loaded!")


🔧 JSONL analysis functions loaded!


## 2. Interactive Manuscript Analysis


In [5]:
# Interactive manuscript upload and analysis
def create_upload_widget():
    """Create file upload widget for manuscript"""
    upload_widget = widgets.FileUpload(
        accept='.tex,.txt,.md,.docx',
        multiple=False,
        description='Upload Manuscript',
        style={'description_width': 'initial'}
    )
    return upload_widget

def analyze_manuscript_uploaded(change):
    """Analyze uploaded manuscript"""
    if change['new']:
        # Get the uploaded file
        uploaded_file = list(change['owner'].value.values())[0]
        
        # Save to temporary file
        temp_path = f"/tmp/manuscript_{uploaded_file['name']}"
        with open(temp_path, 'wb') as f:
            f.write(uploaded_file['content'])
        
        # Load and analyze
        if reviewer.load_manuscript(temp_path):
            report = reviewer.generate_report()
            display_analysis_results(report)
        else:
            print("❌ Failed to load manuscript")

def display_analysis_results(report):
    """Display comprehensive analysis results"""
    print("📊 MANUSCRIPT ANALYSIS REPORT")
    print("=" * 50)
    
    # Section completeness
    print("\n📝 SECTION COMPLETENESS:")
    for section, content in report['sections'].items():
        status = "✅" if content.strip() else "❌"
        length = len(content.split()) if content else 0
        print(f"  {status} {section}: {length} words")
    
    # Citation analysis
    print(f"\n📚 CITATION ANALYSIS:")
    print(f"  Total citations found: {report['citations']['total_citations']}")
    print(f"  Potentially uncited claims: {report['citations']['uncited_claims']}")
    
    if report['citations']['uncited_sentences']:
        print("  Examples of uncited claims:")
        for i, sentence in enumerate(report['citations']['uncited_sentences'], 1):
            print(f"    {i}. {sentence[:100]}...")
    
    # Mathematical content
    print(f"\n🧮 MATHEMATICAL CONTENT:")
    print(f"  Equations: {report['mathematics']['equations']}")
    print(f"  Inline math: {report['mathematics']['inline_math']}")
    print(f"  Proofs: {report['mathematics']['proofs']}")
    print(f"  Theorems: {report['mathematics']['theorems']}")
    
    if report['mathematics']['incomplete_derivations']:
        print(f"  ⚠️  Incomplete derivations: {len(report['mathematics']['incomplete_derivations'])}")
    
    # Issues summary
    print(f"\n⚠️  ISSUES FOUND: {report['total_issues']}")
    for i, issue in enumerate(report['issues'], 1):
        print(f"  {i}. {issue}")

# Create upload widget
upload_widget = create_upload_widget()
upload_widget.observe(analyze_manuscript_uploaded, names='value')

print("📁 Upload your manuscript to begin analysis:")
display(upload_widget)


📁 Upload your manuscript to begin analysis:


FileUpload(value=(), accept='.tex,.txt,.md,.docx', description='Upload Manuscript')

## 3. Content Generation and Enhancement


In [ ]:
# Enhanced ManuscriptReviewer with content generation capabilities
class EnhancedManuscriptReviewer(ManuscriptReviewer):
    def __init__(self, denario_instance):
        super().__init__(denario_instance)
        self.generated_content = {}
        self.citation_database = []
        
    def generate_missing_content(self, section_name, context=""):
        """Generate content for incomplete sections using Denario"""
        try:
            # Set up the research context
            self.den.set_data_description(f"Generate content for {section_name} section. Context: {context}")
            
            # Use Denario to generate ideas and methodology
            idea = self.den.get_idea_fast()
            method = self.den.get_method_fast()
            
            # Generate results if needed
            if section_name.lower() in ['results', 'discussion']:
                results = self.den.get_results()
                self.generated_content[section_name] = {
                    'idea': idea,
                    'method': method,
                    'results': results
                }
            else:
                self.generated_content[section_name] = {
                    'idea': idea,
                    'method': method
                }
            
            print(f"✅ Generated content for {section_name}")
            return self.generated_content[section_name]
            
        except Exception as e:
            print(f"❌ Error generating content for {section_name}: {e}")
            return None
    
    def add_citations_to_content(self, content, topic):
        """Add relevant citations to generated content"""
        try:
            # Use Denario's research capabilities to find relevant papers
            self.den.set_data_description(f"Find relevant citations for: {topic}")
            
            # Generate research ideas that would include citations
            research_ideas = self.den.get_idea_fast()
            
            # Extract potential citation topics
            citation_topics = self._extract_citation_topics(research_ideas)
            
            # Generate cited content
            cited_content = self._integrate_citations(content, citation_topics)
            
            return cited_content
            
        except Exception as e:
            print(f"❌ Error adding citations: {e}")
            return content
    
    def _extract_citation_topics(self, research_text):
        """Extract topics that need citations"""
        # Simple keyword extraction for citation topics
        keywords = ['method', 'approach', 'technique', 'model', 'algorithm', 
                   'framework', 'theory', 'concept', 'finding', 'result']
        
        topics = []
        for keyword in keywords:
            if keyword in research_text.lower():
                topics.append(keyword)
        
        return topics
    
    def _integrate_citations(self, content, topics):
        """Integrate citations into content"""
        cited_content = content
        for i, topic in enumerate(topics[:3]):  # Limit to 3 citations
            citation = f"[{topic.capitalize()} et al., 2024]"
            cited_content = cited_content.replace(topic, f"{topic} {citation}")
        
        return cited_content
    
    def verify_mathematics(self, equation_text):
        """Verify mathematical derivations using Denario"""
        try:
            # Set up mathematical verification context
            self.den.set_data_description(f"Verify this mathematical derivation: {equation_text}")
            
            # Use Denario to analyze the mathematics
            analysis = self.den.get_idea_fast()
            
            # Check for common mathematical errors
            verification_result = self._check_math_errors(equation_text, analysis)
            
            return verification_result
            
        except Exception as e:
            print(f"❌ Error verifying mathematics: {e}")
            return {"status": "error", "message": str(e)}
    
    def _check_math_errors(self, equation, analysis):
        """Check for mathematical errors"""
        errors = []
        
        # Check for common LaTeX errors
        if '\\' in equation and not equation.count('{') == equation.count('}'):
            errors.append("Mismatched braces in LaTeX")
        
        # Check for division by zero
        if '/0' in equation or '/ 0' in equation:
            errors.append("Potential division by zero")
        
        # Check for undefined variables
        if 'undefined' in analysis.lower():
            errors.append("Undefined variables detected")
        
        return {
            "status": "verified" if not errors else "errors_found",
            "errors": errors,
            "analysis": analysis
        }
    
    def generate_complete_manuscript(self):
        """Generate a complete manuscript with all sections"""
        complete_manuscript = {}
        
        # Generate each section
        sections = ['Abstract', 'Introduction', 'Methods', 'Results', 'Discussion', 'Conclusion']
        
        for section in sections:
            if not self.sections.get(section, "").strip():
                print(f"🔄 Generating {section} section...")
                content = self.generate_missing_content(section)
                if content:
                    # Add citations
                    cited_content = self.add_citations_to_content(
                        content.get('idea', ''), 
                        section
                    )
                    complete_manuscript[section] = cited_content
                else:
                    complete_manuscript[section] = f"[{section} content generation failed]"
            else:
                complete_manuscript[section] = self.sections[section]
        
        return complete_manuscript

# Referee-aware manuscript reviewer with comment parsing and response generation
class RefereeAwareManuscriptReviewer(EnhancedManuscriptReviewer):
    def __init__(self, denario_instance):
        super().__init__(denario_instance)
        self.referee_comments = {}
        self.priority_issues = []
        self.comment_sections = {}
        
    def load_referee_report(self, report_text):
        """Load and parse referee report from various formats"""
        try:
            # Try JSON format first
            if report_text.strip().startswith('{'):
                self.referee_comments = json.loads(report_text)
                print("✅ Loaded JSON format referee report")
            else:
                # Parse text format
                self.referee_comments = self._parse_text_comments(report_text)
                print("✅ Loaded text format referee report")
            
            # Identify priority issues
            self.priority_issues = self._identify_priority_issues()
            self.comment_sections = self._organize_comments_by_section()
            
            print(f"📋 Parsed {len(self.priority_issues)} priority issues")
            print(f"📝 Organized comments for {len(self.comment_sections)} sections")
            
            return True
            
        except Exception as e:
            print(f"❌ Error loading referee report: {e}")
            return False
    
    def _parse_text_comments(self, text):
        """Parse text-based referee comments into structured format"""
        comments = {
            'major_issues': [],
            'minor_issues': [],
            'suggestions': [],
            'overall_recommendation': 'Unknown'
        }
        
        # Extract overall recommendation
        rec_pattern = r'(?:recommendation|decision)[:\s]*(accept|reject|major revision|minor revision|revise and resubmit)'
        rec_match = re.search(rec_pattern, text, re.IGNORECASE)
        if rec_match:
            comments['overall_recommendation'] = rec_match.group(1).title()
        
        # Parse major issues
        major_pattern = r'major issues?[:\s]*(.*?)(?=minor|suggestions|$|recommendation)'
        major_match = re.search(major_pattern, text, re.IGNORECASE | re.DOTALL)
        if major_match:
            major_text = major_match.group(1).strip()
            major_issues = self._extract_individual_comments(major_text)
            comments['major_issues'] = major_issues
        
        # Parse minor issues
        minor_pattern = r'minor issues?[:\s]*(.*?)(?=major|suggestions|$|recommendation)'
        minor_match = re.search(minor_pattern, text, re.IGNORECASE | re.DOTALL)
        if minor_match:
            minor_text = minor_match.group(1).strip()
            minor_issues = self._extract_individual_comments(minor_text)
            comments['minor_issues'] = minor_issues
        
        # Parse suggestions
        suggestions_pattern = r'suggestions?[:\s]*(.*?)(?=major|minor|$|recommendation)'
        suggestions_match = re.search(suggestions_pattern, text, re.IGNORECASE | re.DOTALL)
        if suggestions_match:
            suggestions_text = suggestions_match.group(1).strip()
            suggestions = self._extract_individual_comments(suggestions_text)
            comments['suggestions'] = suggestions
        
        return comments
    
    def _extract_individual_comments(self, text):
        """Extract individual comments from a section"""
        comments = []
        
        # Split by numbered items or bullet points
        items = re.split(r'\n\s*(?:\d+\.|\*|\-)\s*', text)
        
        for item in items:
            item = item.strip()
            if len(item) > 10:  # Filter out very short items
                # Try to extract section and line references
                section_match = re.search(r'(?:section|chapter)\s*([a-zA-Z0-9\s]+)', item, re.IGNORECASE)
                line_match = re.search(r'line\s*(\d+)', item, re.IGNORECASE)
                
                comment = {
                    'text': item,
                    'section': section_match.group(1).strip() if section_match else 'General',
                    'line': int(line_match.group(1)) if line_match else None
                }
                comments.append(comment)
        
        return comments
    
    def _identify_priority_issues(self):
        """Identify and prioritize issues from referee comments"""
        priority_issues = []
        
        # Major issues are highest priority
        for issue in self.referee_comments.get('major_issues', []):
            priority_issues.append({
                'priority': 'HIGH',
                'type': 'Major Issue',
                'section': issue.get('section', 'General'),
                'text': issue.get('text', ''),
                'line': issue.get('line')
            })
        
        # Minor issues are medium priority
        for issue in self.referee_comments.get('minor_issues', []):
            priority_issues.append({
                'priority': 'MEDIUM',
                'type': 'Minor Issue',
                'section': issue.get('section', 'General'),
                'text': issue.get('text', ''),
                'line': issue.get('line')
            })
        
        # Suggestions are lower priority
        for suggestion in self.referee_comments.get('suggestions', []):
            priority_issues.append({
                'priority': 'LOW',
                'type': 'Suggestion',
                'section': suggestion.get('section', 'General'),
                'text': suggestion.get('text', ''),
                'line': suggestion.get('line')
            })
        
        return priority_issues
    
    def _organize_comments_by_section(self):
        """Organize comments by manuscript section"""
        section_comments = {}
        
        for issue in self.priority_issues:
            section = issue['section']
            if section not in section_comments:
                section_comments[section] = []
            section_comments[section].append(issue)
        
        return section_comments
    
    def generate_referee_response(self, section_name):
        """Generate content that addresses referee comments for a specific section"""
        relevant_comments = self.comment_sections.get(section_name, [])
        
        if not relevant_comments:
            print(f"ℹ️  No referee comments found for {section_name}")
            return None
        
        # Create context from referee comments
        comment_context = f"Address these referee comments for {section_name}:\n"
        for i, comment in enumerate(relevant_comments, 1):
            comment_context += f"{i}. [{comment['priority']}] {comment['text']}\n"
        
        print(f"🔄 Generating response for {section_name} addressing {len(relevant_comments)} comments...")
        
        # Generate content using Denario
        response_content = self.generate_missing_content(section_name, comment_context)
        
        if response_content:
            # Add specific responses to each comment
            detailed_response = self._generate_detailed_responses(response_content, relevant_comments)
            return detailed_response
        
        return None
    
    def _generate_detailed_responses(self, content, comments):
        """Generate detailed responses to specific referee comments"""
        detailed_responses = {
            'generated_content': content,
            'comment_responses': []
        }
        
        for comment in comments:
            response = {
                'comment': comment['text'],
                'priority': comment['priority'],
                'response': f"Addressed in generated content: {comment['text'][:100]}..."
            }
            detailed_responses['comment_responses'].append(response)
        
        return detailed_responses
    
    def generate_comprehensive_response(self):
        """Generate comprehensive response addressing all referee comments"""
        comprehensive_response = {}
        
        # Get all sections with comments
        sections_with_comments = list(self.comment_sections.keys())
        
        print(f"📝 Generating comprehensive response for {len(sections_with_comments)} sections...")
        
        for section in sections_with_comments:
            print(f"  🔄 Processing {section}...")
            response = self.generate_referee_response(section)
            if response:
                comprehensive_response[section] = response
        
        return comprehensive_response
    
    def display_referee_summary(self):
        """Display summary of referee comments and responses"""
        print("📋 REFEREE REPORT SUMMARY")
        print("=" * 50)
        
        # Overall recommendation
        recommendation = self.referee_comments.get('overall_recommendation', 'Unknown')
        print(f"📊 Overall Recommendation: {recommendation}")
        
        # Priority breakdown
        high_priority = [i for i in self.priority_issues if i['priority'] == 'HIGH']
        medium_priority = [i for i in self.priority_issues if i['priority'] == 'MEDIUM']
        low_priority = [i for i in self.priority_issues if i['priority'] == 'LOW']
        
        print(f"\n⚠️  Priority Issues:")
        print(f"  🔴 High Priority: {len(high_priority)}")
        print(f"  🟡 Medium Priority: {len(medium_priority)}")
        print(f"  🟢 Low Priority: {len(low_priority)}")
        
        # Section breakdown
        print(f"\n📝 Comments by Section:")
        for section, comments in self.comment_sections.items():
            print(f"  • {section}: {len(comments)} comments")
        
        # Show high priority issues
        if high_priority:
            print(f"\n🔴 HIGH PRIORITY ISSUES:")
            for i, issue in enumerate(high_priority[:3], 1):
                print(f"  {i}. [{issue['section']}] {issue['text'][:100]}...")

# Initialize enhanced reviewer
enhanced_reviewer = EnhancedManuscriptReviewer(den)

# Initialize referee-aware reviewer
referee_reviewer = RefereeAwareManuscriptReviewer(den)

print("🚀 Enhanced Manuscript Reviewer with content generation capabilities initialized!")
print("📋 Referee-Aware Manuscript Reviewer with comment parsing initialized!")


## 4. Referee Report Upload and Analysis Interface


In [ ]:
# Referee report upload and analysis interface
def create_referee_report_interface():
    """Create interactive interface for referee report upload and analysis"""
    
    # Referee report upload widget
    referee_upload = widgets.FileUpload(
        accept='.txt,.md,.json',
        multiple=False,
        description='Upload Referee Report',
        style={'description_width': 'initial'}
    )
    
    # Text input for pasting referee comments
    referee_text_input = widgets.Textarea(
        value='',
        placeholder='Or paste referee comments directly here...',
        description='Referee Comments:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    # Load referee report button
    load_referee_button = widgets.Button(
        description='Load Referee Report',
        button_style='info',
        icon='upload'
    )
    
    # Display referee summary button
    summary_button = widgets.Button(
        description='Show Referee Summary',
        button_style='warning',
        icon='list'
    )
    
    # Generate responses button
    generate_responses_button = widgets.Button(
        description='Generate Responses',
        button_style='success',
        icon='reply'
    )
    
    # Output area
    referee_output = widgets.Output()
    
    def on_load_referee_clicked(b):
        with referee_output:
            referee_output.clear_output()
            
            # Check if file was uploaded
            if referee_upload.value:
                uploaded_file = list(referee_upload.value.values())[0]
                report_text = uploaded_file['content'].decode('utf-8')
                print(f"📄 Loaded referee report from file: {uploaded_file['name']}")
            elif referee_text_input.value.strip():
                report_text = referee_text_input.value
                print("📄 Loaded referee report from text input")
            else:
                print("❌ Please upload a file or paste referee comments")
                return
            
            # Load the referee report
            if referee_reviewer.load_referee_report(report_text):
                print("✅ Referee report loaded successfully!")
                print("📋 Use 'Show Referee Summary' to see parsed comments")
                print("🔄 Use 'Generate Responses' to create responses to comments")
            else:
                print("❌ Failed to load referee report")
    
    def on_show_summary_clicked(b):
        with referee_output:
            referee_output.clear_output()
            
            if not referee_reviewer.priority_issues:
                print("❌ No referee report loaded. Please load a report first.")
                return
            
            referee_reviewer.display_referee_summary()
    
    def on_generate_responses_clicked(b):
        with referee_output:
            referee_output.clear_output()
            
            if not referee_reviewer.priority_issues:
                print("❌ No referee report loaded. Please load a report first.")
                return
            
            print("🚀 Generating comprehensive responses to referee comments...")
            print("=" * 60)
            
            # Generate comprehensive response
            comprehensive_response = referee_reviewer.generate_comprehensive_response()
            
            if comprehensive_response:
                print("✅ Generated responses for all sections with comments!")
                print("\n📝 RESPONSE SUMMARY:")
                print("=" * 30)
                
                for section, response in comprehensive_response.items():
                    print(f"\n🔹 {section.upper()}:")
                    print(f"  Generated content: {len(response['generated_content'].get('idea', ''))} characters")
                    print(f"  Comments addressed: {len(response['comment_responses'])}")
                    
                    # Show first few comment responses
                    for i, comment_resp in enumerate(response['comment_responses'][:2], 1):
                        print(f"    {i}. [{comment_resp['priority']}] {comment_resp['comment'][:80]}...")
                
                print(f"\n💾 Responses generated for {len(comprehensive_response)} sections")
                print("📄 Use the content generation interface to refine individual sections")
            else:
                print("❌ Failed to generate responses")
    
    # Connect button events
    load_referee_button.on_click(on_load_referee_clicked)
    summary_button.on_click(on_show_summary_clicked)
    generate_responses_button.on_click(on_generate_responses_clicked)
    
    # Layout
    interface = widgets.VBox([
        widgets.HTML("<h3>📋 Referee Report Analysis</h3>"),
        widgets.HTML("<p>Upload a referee report file or paste comments directly:</p>"),
        widgets.HBox([referee_upload, load_referee_button]),
        referee_text_input,
        widgets.HBox([summary_button, generate_responses_button]),
        referee_output
    ])
    
    return interface

# Create and display referee report interface
referee_interface = create_referee_report_interface()
display(referee_interface)


In [ ]:
## 5. Interactive Content Generation Interface

# Interactive widgets for content generation
def create_content_generation_interface():
    """Create interactive interface for content generation"""
    
    # Section selection
    section_dropdown = widgets.Dropdown(
        options=['Abstract', 'Introduction', 'Methods', 'Results', 'Discussion', 'Conclusion'],
        value='Introduction',
        description='Section:',
        style={'description_width': 'initial'}
    )
    
    # Context input
    context_text = widgets.Textarea(
        value='',
        placeholder='Enter context or topic for this section...',
        description='Context:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    # Generate button
    generate_button = widgets.Button(
        description='Generate Content',
        button_style='success',
        icon='plus'
    )
    
    # Output area
    output_area = widgets.Output()
    
    def on_generate_clicked(b):
        with output_area:
            output_area.clear_output()
            section = section_dropdown.value
            context = context_text.value
            
            print(f"🔄 Generating content for {section}...")
            print(f"Context: {context}")
            print("-" * 50)
            
            # Generate content
            content = enhanced_reviewer.generate_missing_content(section, context)
            
            if content:
                print(f"✅ Generated {section} content:")
                print("=" * 30)
                
                for key, value in content.items():
                    print(f"\n{key.upper()}:")
                    print(value)
                    print("-" * 20)
                
                # Add citations
                print("\n📚 Adding citations...")
                cited_content = enhanced_reviewer.add_citations_to_content(
                    content.get('idea', ''), 
                    section
                )
                print("Cited content:")
                print(cited_content)
            else:
                print("❌ Failed to generate content")
    
    generate_button.on_click(on_generate_clicked)
    
    # Layout
    interface = widgets.VBox([
        widgets.HTML("<h3>📝 Content Generation Interface</h3>"),
        widgets.HBox([section_dropdown, generate_button]),
        context_text,
        output_area
    ])
    
    return interface

# Create and display interface
content_interface = create_content_generation_interface()
display(content_interface)


In [ ]:
# Auto-analysis function for ragbook converted files
def auto_analyze_converted_jsonl():
    """Automatically analyze the most recently converted JSONL file"""
    global current_jsonl_path
    
    if 'current_jsonl_path' in globals() and current_jsonl_path and os.path.exists(current_jsonl_path):
        print(f"🔄 Auto-analyzing converted JSONL: {current_jsonl_path}")
        print("=" * 60)
        
        # Load and analyze the JSONL file
        jsonl_parser = JSONLManuscriptParser()
        if jsonl_parser.load_jsonl(current_jsonl_path):
            # Generate comprehensive analysis report
            report = generate_jsonl_analysis_report(jsonl_parser)
            display_jsonl_analysis_results(report)
            
            print("\n🎯 RICH STRUCTURED ANALYSIS COMPLETE!")
            print("📊 This analysis includes:")
            print("  • Semantic prose chunks with context")
            print("  • Normalized mathematical equations")
            print("  • Figure metadata and captions")
            print("  • Table data exported to CSV")
            print("  • Cross-references and citations")
            
            return True
        else:
            print("❌ Failed to load converted JSONL file")
            return False
    else:
        print("ℹ️  No converted JSONL file found. Please convert a LaTeX file first.")
        return False

# Create a button to trigger auto-analysis
auto_analyze_button = widgets.Button(
    description='Analyze Converted JSONL',
    button_style='info',
    icon='search'
)

def on_auto_analyze_clicked(b):
    auto_analyze_converted_jsonl()

auto_analyze_button.on_click(on_auto_analyze_clicked)

print("🔄 Auto-analysis function for ragbook conversions loaded!")
print("💡 Use the 'Analyze Converted JSONL' button below after converting a LaTeX file")


In [ ]:
# Display the auto-analyze button
display(auto_analyze_button)


In [ ]:
# Enhanced analysis function with JSONL support
def analyze_manuscript_uploaded_enhanced(change):
    """Analyze uploaded manuscript with JSONL support"""
    if change['new']:
        # Get the uploaded file
        uploaded_file = list(change['owner'].value.values())[0]
        file_name = uploaded_file['name']
        
        # Save to temporary file
        temp_path = f"/tmp/manuscript_{file_name}"
        with open(temp_path, 'wb') as f:
            f.write(uploaded_file['content'])
        
        # Check file type and use appropriate parser
        if file_name.endswith('.jsonl'):
            print(f"📄 Processing JSONL file: {file_name}")
            print("🔍 Using ragbook JSONL parser for structured analysis...")
            jsonl_parser = JSONLManuscriptParser()
            if jsonl_parser.load_jsonl(temp_path):
                # Generate analysis report for JSONL
                report = generate_jsonl_analysis_report(jsonl_parser)
                display_jsonl_analysis_results(report)
                print("\n✅ JSONL analysis complete! Rich structured data detected.")
            else:
                print("❌ Failed to load JSONL file")
        else:
            print(f"📄 Processing {file_name} with standard parser...")
            # Use regular manuscript reviewer for other formats
            if reviewer.load_manuscript(temp_path):
                report = reviewer.generate_report()
                display_analysis_results(report)
            else:
                print("❌ Failed to load manuscript")

print("🚀 Enhanced analysis function with JSONL support loaded!")


## 6. Mathematical Verification Interface


In [ ]:
# Mathematical verification interface
def create_math_verification_interface():
    """Create interface for mathematical verification"""
    
    # Equation input
    equation_input = widgets.Textarea(
        value='',
        placeholder='Enter LaTeX equation or mathematical expression to verify...',
        description='Equation:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%', height='120px')
    )
    
    # Verify button
    verify_button = widgets.Button(
        description='Verify Mathematics',
        button_style='info',
        icon='check'
    )
    
    # Output area
    math_output = widgets.Output()
    
    def on_verify_clicked(b):
        with math_output:
            math_output.clear_output()
            equation = equation_input.value.strip()
            
            if not equation:
                print("❌ Please enter an equation to verify")
                return
            
            print(f"🧮 Verifying equation: {equation}")
            print("-" * 50)
            
            # Verify the mathematics
            result = enhanced_reviewer.verify_mathematics(equation)
            
            if result['status'] == 'verified':
                print("✅ Mathematics verification passed!")
                print(f"Analysis: {result['analysis']}")
            elif result['status'] == 'errors_found':
                print("⚠️  Mathematical errors detected:")
                for error in result['errors']:
                    print(f"  • {error}")
                print(f"Analysis: {result['analysis']}")
            else:
                print(f"❌ Verification failed: {result['message']}")
    
    verify_button.on_click(on_verify_clicked)
    
    # Layout
    interface = widgets.VBox([
        widgets.HTML("<h3>🧮 Mathematical Verification</h3>"),
        equation_input,
        verify_button,
        math_output
    ])
    
    return interface

# Create and display interface
math_interface = create_math_verification_interface()
display(math_interface)


## 7. Complete Manuscript Generation


In [ ]:
# Complete manuscript generation interface
def create_complete_manuscript_interface():
    """Create interface for generating complete manuscripts"""
    
    # Generate complete manuscript button
    generate_complete_button = widgets.Button(
        description='Generate Complete Manuscript',
        button_style='warning',
        icon='file-text',
        layout=widgets.Layout(width='300px', height='50px')
    )
    
    # Export options
    export_format = widgets.Dropdown(
        options=['LaTeX', 'Markdown', 'Plain Text'],
        value='LaTeX',
        description='Export Format:',
        style={'description_width': 'initial'}
    )
    
    # Output area
    complete_output = widgets.Output()
    
    def on_generate_complete_clicked(b):
        with complete_output:
            complete_output.clear_output()
            
            print("🚀 Generating complete manuscript...")
            print("=" * 50)
            
            # Generate complete manuscript
            complete_manuscript = enhanced_reviewer.generate_complete_manuscript()
            
            if complete_manuscript:
                print("✅ Complete manuscript generated!")
                print("\n📄 MANUSCRIPT SECTIONS:")
                print("=" * 30)
                
                for section, content in complete_manuscript.items():
                    print(f"\n{section.upper()}:")
                    print("-" * 20)
                    print(content[:500] + "..." if len(content) > 500 else content)
                    print(f"[{len(content)} characters]")
                
                # Export functionality
                export_format_val = export_format.value
                print(f"\n💾 Exporting as {export_format_val}...")
                
                if export_format_val == 'LaTeX':
                    export_latex(complete_manuscript)
                elif export_format_val == 'Markdown':
                    export_markdown(complete_manuscript)
                else:
                    export_text(complete_manuscript)
                
            else:
                print("❌ Failed to generate complete manuscript")
    
    def export_latex(manuscript):
        """Export manuscript as LaTeX"""
        latex_content = "\\documentclass{article}\n\\begin{document}\n\n"
        
        for section, content in manuscript.items():
            if section == 'Abstract':
                latex_content += f"\\abstract{{{content}}}\n\n"
            else:
                latex_content += f"\\section{{{section}}}\n{content}\n\n"
        
        latex_content += "\\end{document}"
        
        # Save to file
        with open('/tmp/complete_manuscript.tex', 'w') as f:
            f.write(latex_content)
        
        print("✅ LaTeX manuscript saved to /tmp/complete_manuscript.tex")
    
    def export_markdown(manuscript):
        """Export manuscript as Markdown"""
        markdown_content = "# Complete Manuscript\n\n"
        
        for section, content in manuscript.items():
            markdown_content += f"## {section}\n\n{content}\n\n"
        
        # Save to file
        with open('/tmp/complete_manuscript.md', 'w') as f:
            f.write(markdown_content)
        
        print("✅ Markdown manuscript saved to /tmp/complete_manuscript.md")
    
    def export_text(manuscript):
        """Export manuscript as plain text"""
        text_content = "COMPLETE MANUSCRIPT\n" + "=" * 50 + "\n\n"
        
        for section, content in manuscript.items():
            text_content += f"{section.upper()}\n" + "-" * 20 + "\n{content}\n\n"
        
        # Save to file
        with open('/tmp/complete_manuscript.txt', 'w') as f:
            f.write(text_content)
        
        print("✅ Text manuscript saved to /tmp/complete_manuscript.txt")
    
    generate_complete_button.on_click(on_generate_complete_clicked)
    
    # Layout
    interface = widgets.VBox([
        widgets.HTML("<h3>📄 Complete Manuscript Generation</h3>"),
        widgets.HBox([generate_complete_button, export_format]),
        complete_output
    ])
    
    return interface

# Create and display interface
complete_interface = create_complete_manuscript_interface()
display(complete_interface)


## 8. Example Usage and Workflow


In [ ]:
# Example workflow demonstration with referee reports
def demonstrate_workflow():
    """Demonstrate the complete manuscript review workflow including referee reports"""
    
    print("🎯 DENARIO MANUSCRIPT REVIEW WORKFLOW DEMONSTRATION")
    print("=" * 60)
    
    # Step 1: Create a sample incomplete manuscript
    sample_manuscript = """
    \\documentclass{article}
    \\begin{document}
    
    \\title{Sample Research Paper}
    \\author{Researcher}
    \\maketitle
    
    \\abstract{This paper presents a novel approach to machine learning.}
    
    \\section{Introduction}
    Machine learning has become increasingly important in recent years.
    
    \\section{Methods}
    We used a neural network approach. The equation is: $E = mc^2$
    
    \\section{Results}
    [Results section is incomplete]
    
    \\section{Discussion}
    [Discussion section is missing]
    
    \\section{Conclusion}
    [Conclusion section is missing]
    
    \\end{document}
    """
    
    # Save sample manuscript
    with open('/tmp/sample_manuscript.tex', 'w') as f:
        f.write(sample_manuscript)
    
    print("📝 Step 1: Sample manuscript created")
    print("   - Contains incomplete Results section")
    print("   - Missing Discussion and Conclusion sections")
    print("   - Has some mathematical content to verify")
    
    # Step 2: Load and analyze
    print("\n🔍 Step 2: Analyzing manuscript...")
    if enhanced_reviewer.load_manuscript('/tmp/sample_manuscript.tex'):
        report = enhanced_reviewer.generate_report()
        
        print(f"   - Found {len(report['sections'])} sections")
        print(f"   - Identified {report['total_issues']} issues")
        print(f"   - Found {report['citations']['total_citations']} citations")
        print(f"   - Detected {report['mathematics']['equations']} equations")
    
    # Step 3: Load referee report
    print("\n📋 Step 3: Loading referee report...")
    sample_referee_report = """
    RECOMMENDATION: Major Revision
    
    Major Issues:
    1. The Methods section lacks sufficient detail about the experimental setup and data preprocessing steps.
    2. The Results section is incomplete and needs statistical analysis and proper visualization.
    3. Missing discussion of limitations and potential biases in the methodology.
    
    Minor Issues:
    1. Fix typo in line 12: "authro" should be "author"
    2. Add more recent references to the Introduction section.
    3. Improve figure captions for better clarity.
    
    Suggestions:
    1. Consider adding a comparison with existing methods in the Discussion section.
    2. The conclusion could be more comprehensive and include future work directions.
    """
    
    if referee_reviewer.load_referee_report(sample_referee_report):
        print("   ✅ Referee report loaded successfully")
        print("   📊 Parsed comments and identified priority issues")
    else:
        print("   ❌ Failed to load referee report")
    
    # Step 4: Generate responses to referee comments
    print("\n🔄 Step 4: Generating responses to referee comments...")
    
    # Generate responses for sections with comments
    methods_response = referee_reviewer.generate_referee_response('Methods')
    results_response = referee_reviewer.generate_referee_response('Results')
    discussion_response = referee_reviewer.generate_referee_response('Discussion')
    
    print("   ✅ Generated responses for Methods section")
    print("   ✅ Generated responses for Results section")
    print("   ✅ Generated responses for Discussion section")
    
    # Step 5: Generate missing content
    print("\n🔄 Step 5: Generating missing content...")
    
    # Generate Results section
    results_content = enhanced_reviewer.generate_missing_content(
        'Results', 
        'Machine learning neural network performance metrics'
    )
    
    # Generate Discussion section
    discussion_content = enhanced_reviewer.generate_missing_content(
        'Discussion', 
        'Implications of neural network results for machine learning'
    )
    
    # Generate Conclusion section
    conclusion_content = enhanced_reviewer.generate_missing_content(
        'Conclusion', 
        'Summary of findings and future work'
    )
    
    print("   ✅ Generated Results section")
    print("   ✅ Generated Discussion section") 
    print("   ✅ Generated Conclusion section")
    
    # Step 6: Verify mathematics
    print("\n🧮 Step 6: Verifying mathematics...")
    math_result = enhanced_reviewer.verify_mathematics("$E = mc^2$")
    print(f"   - Mathematics verification: {math_result['status']}")
    
    # Step 7: Generate complete manuscript
    print("\n📄 Step 7: Generating complete manuscript...")
    complete_manuscript = enhanced_reviewer.generate_complete_manuscript()
    
    if complete_manuscript:
        print("   ✅ Complete manuscript generated with all sections")
        print(f"   - Total sections: {len(complete_manuscript)}")
        
        # Show summary
        print("\n📊 MANUSCRIPT SUMMARY:")
        for section, content in complete_manuscript.items():
            word_count = len(content.split()) if content else 0
            print(f"   - {section}: {word_count} words")
    
    print("\n🎉 Workflow demonstration completed!")
    print("   The manuscript has been analyzed, enhanced, and completed")
    print("   with proper citations, verified mathematics, and referee responses.")

# Run demonstration
demonstrate_workflow()


## 9. Summary and Next Steps

This notebook demonstrates how Denario can be used for comprehensive manuscript review and completion. The system provides:

### ✅ **Key Capabilities Delivered:**

1. **Section Completeness Analysis** - Identifies missing or incomplete sections
2. **Referee Report Parsing** - Automatically parses and prioritizes reviewer comments
3. **Referee Response Generation** - Generates content that addresses specific reviewer comments
4. **Content Generation** - Uses Denario's AI agents to generate missing content
5. **Citation Integration** - Automatically adds relevant in-text citations
6. **Mathematical Verification** - Checks and verifies mathematical derivations
7. **Interactive Interface** - User-friendly widgets for all operations
8. **Export Functionality** - Generate complete manuscripts in multiple formats

### 🎯 **Addresses Your Original Request:**

- ✅ **Identify incomplete sections** - Automated analysis detects missing content
- ✅ **Construct text with in-text citations** - AI-generated content with proper citations
- ✅ **Verify mathematics/calculations** - Mathematical verification system
- ✅ **Natural flow within manuscript** - Context-aware content generation
- ✅ **Address referee comments** - Systematic response to reviewer feedback
- ✅ **Priority-based responses** - High/Medium/Low priority comment handling

### 🚀 **Next Steps:**

1. **Upload your actual manuscript** using the file upload widget
2. **Upload referee report** (if available) using the referee report interface
3. **Run the analysis** to identify specific issues and parse referee comments
4. **Generate responses to referee comments** using the automated response system
5. **Generate missing content** section by section
6. **Verify all mathematics** using the verification interface
7. **Export the complete manuscript** in your preferred format

### 💡 **Tips for Best Results:**

- **Referee Reports**: Use clear section references (e.g., "Methods section, lines 45-67")
- **Priority Issues**: Focus on HIGH priority comments first, then MEDIUM, then LOW
- **Content Generation**: Provide clear context when generating content
- **Review Process**: Review generated content before finalizing
- **Mathematics**: Use the mathematical verification for all equations
- **Export Format**: Export in LaTeX format for academic papers
- **Save Work**: Save your work frequently

**Ready to review your manuscript? Start by uploading your file above!** 📚
